In [42]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import export_graphviz
%matplotlib inline

In [43]:
train = pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\train.csv")
test =  pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\test.csv")

In [50]:
g_Apartment_col=train.groupby(['X0','X314'])['y'].agg(['mean','count'])
g_Apartment_col

,,mean,count
X0,X314,,
a,0,115.630952,21
aa,0,152.170000,2
ab,0,112.300000,1
ac,0,89.620000,1
ad,0,94.087857,14
af,0,112.791143,35
ai,0,95.810588,34
aj,1,103.974834,151
ak,1,112.552235,349


In [27]:
# store it as Y
train=train[train.y<=170]
Y_train = train["y"]
train.drop(["y"], axis=1, inplace=True)
id_test = test["ID"]
train.describe()

,ID,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4208.000000,4208.000000,4208.0,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,...,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000
mean,4206.539686,0.013308,0.0,0.075095,0.057985,0.428232,0.000475,0.002614,0.007605,0.007842,...,0.318916,0.057272,0.314876,0.020675,0.009506,0.008080,0.007605,0.001663,0.000475,0.001426
std,2437.609011,0.114604,0.0,0.263576,0.233742,0.494881,0.021798,0.051067,0.086882,0.088219,...,0.466112,0.232389,0.464522,0.142310,0.097044,0.089535,0.086882,0.040757,0.021798,0.037738
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2095.750000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4220.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6314.250000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8417.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
train.describe(include=["O"])

,X0,X1,X2,X3,X4,X5,X6,X8
count,4208,4208,4208,4208,4208,4208,4208,4208
unique,47,27,44,7,4,29,12,25
top,z,aa,as,c,d,w,g,j
freq,360,833,1659,1942,4204,231,1042,277


In [29]:
num_train = len(train)
all_data = pd.concat([train, test])

In [30]:
all_data.drop(["ID"], axis=1, inplace=True)
all_data = pd.get_dummies(all_data)

In [31]:
X_train = all_data[:num_train]
X_test = all_data[num_train:]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split( X_train, Y_train, test_size = 0.2, random_state = 100)

In [33]:
from sklearn.metrics import r2_score
#print(r2_score(y_pred, y_cv))

In [34]:
d = []
# Decision tree tuning
for crtr in ['mse']:
    for md in [5,10,15,20,25]:
        for spltr in ['best','random']:
            for mss in [2,3,5,10,15,25,40,65,105]:
                for msl in [2,3,5,10,15,25,40,65,105]:
                    dts = DecisionTreeRegressor(criterion=crtr, max_depth=md,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=msl,
                                min_samples_split=mss, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter=spltr)
                    dts.fit(X_train, y_train)
                    y_pred = dts.predict(X_cv)
                    sip=r2_score(y_cv,y_pred)
                    print("score for {} criterion, {} max_depth, {} splitter, {} min_samples_split, {} min_samples_leaf is {}".format(crtr,md,spltr,mss,msl,sip))
                    d.append({'criterion': crtr, 'max_depth': md, 'splitter':spltr , 'min_samples_split' :mss, 'min_samples_leaf' : msl, 'score' : sip})

score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 2 min_samples_leaf is 0.5468724501463902
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 3 min_samples_leaf is 0.553005175272535
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 5 min_samples_leaf is 0.5593287407311601
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 10 min_samples_leaf is 0.5704122694546261
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 15 min_samples_leaf is 0.5644253542111737
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 25 min_samples_leaf is 0.567266321991948
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 40 min_samples_leaf is 0.560563455278255
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 65 min_samples_leaf is 0.5629374582064472
score for mse criterion, 5 max_depth, best splitter, 2 min_samples_split, 105 

In [35]:
d=pd.DataFrame(d)

In [36]:
d.to_csv('bestmodel2.csv', index=False) # LB : 0.74163

In [37]:
dtr_tuned = DecisionTreeRegressor(criterion='mse', max_depth=10,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=15,
                                min_samples_split=65, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter='best')

In [38]:
dtr_tuned.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=15, min_samples_split=65,
           min_weight_fraction_leaf=0.0, presort=False, random_state=100,
           splitter='best')

In [39]:
r2_score(y_cv,dtr_tuned.predict(X_cv))

0.56267557768503973

In [40]:
y_test = dtr_tuned.predict(X_test)
submission = pd.DataFrame({
        "ID": id_test,
        "y": y_test
    })
submission.to_csv('tunedtree.csv', index=False)

In [41]:
# This statement builds a dot file.
cols = list(X_train.columns.values)
tree.export_graphviz(dtr_tuned, out_file='tunedtree.dot',feature_names  = cols) 

In [19]:
# A parameter grid for XGBoost
params = {'criterion':['mse'],  'max_depth': [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,29], 'min_samples_leaf' : [1,2,5,10,15,25,40,65,105] ,
           'min_samples_split' : [2,5,10,15,25,40,65,105] ,'splitter' : ['best','random']}

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
dtr = DecisionTreeRegressor()

In [20]:
grid = GridSearchCV(dtr, params)
grid.fit(X_train, y_train)

KeyboardInterrupt: 

In [14]:
grid.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=5, min_samples_split=105,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='random')

In [15]:
# Print the r2 score
print(r2_score(y_cv, grid.best_estimator_.predict(X_cv))) 

0.578033073721


In [17]:
y_test = grid.best_estimator_.predict(X_test)
submission = pd.DataFrame({
        "ID": id_test,
        "y": y_test
    })
submission.to_csv('gridtunedtree.csv', index=False)

In [19]:
grid.grid_scores_

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.56589, std: 0.05479, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 5, 'splitter': 'best'},
 mean: 0.56589, std: 0.05479, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 5, 'splitter': 'random'},
 mean: 0.56589, std: 0.05479, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 10, 'splitter': 'best'},
 mean: 0.56589, std: 0.05479, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 10, 'splitter': 'random'},
 mean: 0.57481, std: 0.04602, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 15, 'splitter': 'best'},
 mean: 0.56858, std: 0.05348, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 15, 'splitter': 'random'},
 mean: 0.57609, std: 0.04535, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 25, 'splitter

In [ ]:
 mean: 0.58003, std: 0.03733, params: {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 40, 'min_samples_split': 105, 'splitter': 'best'},